This program scrapes a band or artists lyrics from Genius and uses these lyrics generate lyrics using Markov chains and to train a neural network to write lyrics in the style of the artist.

In [ ]:
!pip3 install git+git://github.com/minimaxir/textgenrnn.git#v1.5.0
!pip install lyricsgenius

from lyricsgenius import Genius
import numpy as np
from google.colab import files
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from textgenrnn import textgenrnn
from datetime import datetime
import os

  Cloning git://github.com/minimaxir/textgenrnn.git to /tmp/pip-req-build-gv4cvumt
  Running command git clone -q git://github.com/minimaxir/textgenrnn.git /tmp/pip-req-build-gv4cvumt


This section prompts a user to enter a band or artist name and scrapes all the lyrics from that artist from the website Genius using the lyricsgenius library. [note: It takes a few minutes to scrape this content.]

In [ ]:
artist_name = str(input("Enter the artists name:")) 

genius = Genius('2fThypCYHhkShgZ8ph2qpyaqxVgi5edTgUh4P06_q7AuS5UC9zs7Azdo_rjY-X1U')

genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

artist = genius.search_artist(artist_name, sort="title")

lyrics = artist.save_lyrics(filename='Lyrics_' + artist_name.replace(" ", ""), extension="txt")

Enter the artists name:The Magnetic Fields
Wrote Lyrics_TheMagneticFields.txt.


In [ ]:
lyricStr = open("Lyrics_" + artist_name.replace(" ", "") + ".txt")

lyricStr1 = lyricStr.read().replace("EmbedShare URLCopyEmbedCopy", " ")
lyricStr2 = lyricStr1.replace("\n", " ")

with open("Lyrics_" + artist_name.replace(" ", "") + "_Clean.txt", "w") as text_file:
    text_file.write(lyricStr2)

This section generates lyrics using Markov chains. It starts with a random word and makes a random choice of the next word influenced by the probabilities of a word appearing next in the dataset. 

In [ ]:
def generateLyrics(inspiration, words):
    corpus = inspiration.split()

    def make_pairs(corpus):
        for i in range(len(corpus)-1):
            yield (corpus[i], corpus[i+1])
            
    pairs = make_pairs(corpus)

    word_dict = {}

    for word_1, word_2 in pairs:
        if word_1 in word_dict.keys():
            word_dict[word_1].append(word_2)
        else:
            word_dict[word_1] = [word_2]
     
    first_word = np.random.choice(corpus)

    while first_word.islower():
        first_word = np.random.choice(corpus)

    chain = [first_word]

    n_words = words

    for i in range(n_words):
        chain.append(np.random.choice(word_dict[chain[-1]]))
    
    mc_lyrics = " "
    mc_lyrics = mc_lyrics.join(chain)
    print(mc_lyrics)

generateLyrics(lyricStr1, 100)

But let's make you were young, and abandoned, and Ted And nothing else you let go 'round just for a funny And now I saw you won't get too And nothing I think I want to do It's just stand there, we lived here Your face all I never give our marriage certificate And men First time is done Make a dancing in darkness without the children drowned Now children, listen well and baby Although my name again I've always got strange for a wealthy man grow Plant white In the air You used that make it still getting up with


This section uses the textgenrnn library, a recursive neural network, to generate lyrics. 

See the demo notebook (https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about the parameters in the cell below.

[note: Depending on the parameters chosen, it can take anywhere from a few minutes to a few hours to train the neural network and generate lyrics.]

In [ ]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 2,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 1000000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 20,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
file_name = "Lyrics_"+ artist_name.replace(" ", "") + "_Clean.txt"
model_name = artist_name.replace(" ", "") + '_Output'   # change to set file name of resulting trained models/texts

The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 4-layer, 128-cell Bidirectional LSTMs
Training on 40,113 word sequences.
Epoch 1/20
39/39 [==============================] - 74s 2s/step - loss: 6.8809 - val_loss: 6.2319
Epoch 2/20
39/39 [==============================] - 57s 1s/step - loss: 5.7664 - val_loss: 5.7987
Epoch 3/20
39/39 [==============================] - 57s 1s/step - loss: 5.0983 - val_loss: 5.4214
Epoch 4/20
39/39 [==============================] - 57s 1s/step - loss: 4.4434 - val_loss: 5.3035
Epoch 5/20
39/39 [==============================] - 57s 1s/step - loss: 3.8638 - val_loss: 5.2264
Epoch 6/20
39/39 [==============================] - 58s 1s/step - loss: 3.3182 - val_loss: 5.3126
Epoch 7/20
39/39 [==============================] - 57s 1s/step - loss: 2.8151 - val_loss: 5.4460
Epoch 8/20
39/39 [==============================] - 56s 1s/step - loss: 2.4596 - val_loss: 5.5310
Epoch 9/20
39/39 [==============================] - 57s 1s/step - loss: 2.1847 - val_loss: 5.5598
Epoch 10/20
39/39 [====